<a href="https://colab.research.google.com/github/axel-sirota/neural-networks-image-classification/blob/main/Module3Demo1_TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 34.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [2]:
# Step 1: Import necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from datasets import load_dataset
import os
from PIL import Image
import io


In [3]:
# Step 2: Download and prepare the dataset from Hugging Face
dataset = load_dataset('keremberke/chest-xray-classification', 'full')

# Prepare directories
os.makedirs('xray_dataset/train/normal', exist_ok=True)
os.makedirs('xray_dataset/train/pneumonia', exist_ok=True)
os.makedirs('xray_dataset/validation/normal', exist_ok=True)
os.makedirs('xray_dataset/validation/pneumonia', exist_ok=True)


# # Assuming the dataset splits are already defined in Hugging Face dataset
for split in ['train', 'validation']:
    images = dataset[split]['image']
    labels = dataset[split]['labels']

    for i, (img, label) in enumerate(zip(images, labels)):
        if label == 0:
            label_folder = 'normal'
        else:
            label_folder = 'pneumonia'
        img_path = f'xray_dataset/{split}/{label_folder}/{split}_{label}_{i}.jpeg'
        buffer = io.BytesIO()
        img.save(buffer, format='JPEG')
        with open(img_path, 'wb') as f:
            f.write(buffer.getvalue())


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/4077 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1165 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/582 [00:00<?, ? examples/s]

In [17]:

# Step 3: Preprocess the dataset
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_dir = 'xray_dataset/train'
validation_dir = 'xray_dataset/validation'

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=20,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=20,
    class_mode='binary',
    shuffle=False  # Important for correct label ordering
)


Found 4077 images belonging to 2 classes.
Found 1165 images belonging to 2 classes.


In [18]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import GlobalAveragePooling2D

In [19]:
# Load the pre-trained VGG-19 model, excluding the top layer
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base model
base_model.trainable = False

In [20]:

# Step 4: Build the CNN model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dense(1, activation='sigmoid')  # Assuming binary classification for pneumonia
])

In [21]:


# Step 5: Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 global_average_pooling2d_1  (None, 512)               0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_2 (Dense)             (None, 1024)              525312    
                                                                 
 dense_3 (Dense)             (None, 1)                 1025      
                                                                 
Total params: 20550721 (78.39 MB)
Trainable params: 526337 (2.01 MB)
Non-trainable params: 20024384 (76.39 MB)
_________________________________________________________________


In [23]:

# Step 6: Train the model
history = model.fit(
    train_generator,
    epochs=2,
    validation_data=validation_generator
    )


Epoch 1/2
204/204 [==============================] - 53s 255ms/step - loss: 0.3818 - accuracy: 0.8239 - val_loss: 0.2763 - val_accuracy: 0.8927
Epoch 2/2
204/204 [==============================] - 52s 254ms/step - loss: 0.2809 - accuracy: 0.8793 - val_loss: 0.3198 - val_accuracy: 0.8738


In [24]:
model.evaluate(validation_generator)

59/59 [==============================] - 3s 52ms/step - loss: 0.3198 - accuracy: 0.8738


[0.3197556436061859, 0.8738197684288025]

In [25]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

predictions = model.predict(validation_generator)
predicted_classes = np.where(predictions > 0.5, 1, 0)
true_classes = validation_generator.classes

# Evaluating the model
conf_matrix = confusion_matrix(true_classes, predicted_classes)
report = classification_report(true_classes, predicted_classes, target_names=['Pneumonia', 'Normal'])

# Extracting precision and recall from the classification report
print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(report)

59/59 [==============================] - 3s 50ms/step
Confusion Matrix:
[[289  15]
 [132 729]]

Classification Report:
              precision    recall  f1-score   support

   Pneumonia       0.69      0.95      0.80       304
      Normal       0.98      0.85      0.91       861

    accuracy                           0.87      1165
   macro avg       0.83      0.90      0.85      1165
weighted avg       0.90      0.87      0.88      1165

